In [42]:
import numpy as np
from collections import defaultdict
import queue
mapp = []
with open('Day20.txt') as f:
    for line in f.readlines():
        line = [ord(l) for l in line if l != '\n']
        mapp.append(line)
for i in range(len(mapp[0]) - len(mapp[-1])):
    mapp[-1].append(32)
mapp = np.pad(np.array(mapp),(1,1),mode='constant',constant_values = 32)
#Find all Portals Names & Location
match_index = np.argwhere(mapp >=65).tolist()
matched_list = []
while match_index:
    match1 = match_index.pop()
    match2 = match_index[np.argmin(np.sum(np.abs(np.array(match1) - np.array(match_index)),axis=1))]
    match_index.remove(match2)
    matched_list.append([match1,match2])
#Find all Tiles Next to Portal Names
all_dict = defaultdict(list)
for match in matched_list:
    letter_combo = [mapp[tuple(match[0])],mapp[tuple(match[1])]]
    letter_combo.sort()
    col_row = np.argmax(abs(np.array(match[0]) - np.array(match[1]))) #1 look at column, 0 look at row
    next1s = min(np.array(match)[:,col_row]) - 1
    a = match[0][int(not col_row)]
    if col_row:
        row = a
        col = next1s
        if mapp[row,col] != 46:
            col = next1s + 3
    else:
        row = next1s
        col = a
        if mapp[row,col] != 46:
            row = next1s + 3
    if letter_combo[0] == letter_combo[1]:
        if letter_combo[0] == 65:
            start = tuple((row,col))
        elif letter_combo[0] == 90:
            end = tuple((row,col))
        else:
            all_dict[tuple(letter_combo)].append(tuple((row,col)))
    else:
        all_dict[tuple(letter_combo)].append(tuple((row,col)))
#Create Portal Dictionary
portal_dict = {value[0]:value[1] for value in all_dict.values()}
portal_dict.update({value[1]:value[0] for value in all_dict.values()})
outer_portal = [key for key in portal_dict 
                if ((key[0] == 3) | (key[1]== 3) | (key[0]== (mapp.shape[0]-4))| (key[1]== (mapp.shape[1]-4)))]

In [43]:
#Breadth First Search
all_paths = np.argwhere(mapp == 46)
all_paths = {tuple(xy) for xy in all_paths}
all_paths_count = dict.fromkeys(all_paths,0)
direc = np.array([[0,1],[0,-1],[1,0],[-1,0]])
visited = set()
nextxy = queue.Queue()
curxy = start
nextxy.put(start) #Currently a tuple
while curxy != end:
    curxy = nextxy.get()
    visited.add(curxy)
    neighbors = np.array(curxy) + direc
    neighbors = {tuple(xy) for xy in neighbors}
    neighbors = all_paths.intersection(neighbors) - visited
    for n in neighbors:
        if n in portal_dict:
            n = portal_dict[n]
            all_paths_count[n] = all_paths_count[curxy] + 2
        else:
            all_paths_count[n] = all_paths_count[curxy] + 1
        nextxy.put(n)
all_paths_count[end]

498

In [44]:
#Part 2
all_paths_ori = np.argwhere(mapp == 46).tolist() #Original path index
map_level = 0
all_paths = {tuple([map_level]+xy) for xy in all_paths_ori} #update for each layer added
all_paths_count = dict.fromkeys(all_paths,0) #update for each layer added
end_o, start_o = tuple([map_level] + list(end)), tuple([map_level] + list(start))

In [ ]:
direc = np.array([[0,0,1],[0,0,-1],[0,1,0],[0,-1,0]])
visited = set()
nextxy = queue.Queue()
curxy = start_o
nextxy.put(start_o) #Currently a tuple
while curxy != end_o:
    curxy = nextxy.get()
    visited.add(curxy)
    neighbors = np.array(curxy) + direc
    neighbors = {tuple(xy) for xy in neighbors}
    neighbors = all_paths.intersection(neighbors) - visited
    for n in neighbors:
        nxy = n[1:]
        if nxy in portal_dict:
            nxy1 = portal_dict[nxy]
            if nxy in outer_portal: #go one level back
                n = tuple((n[0]-1,nxy1[0],nxy1[1]))
            else: #not in portal_tracker, go one level deeper
                if map_level < (curxy[0]+1): #If map_level is less than curxy
                    map_level +=1
                    all_path_next = {tuple([map_level]+xy) for xy in all_paths_ori}
                    all_paths.update(all_path_next)
                    all_paths_count.update(dict.fromkeys(all_path_next,0))
                n = tuple((n[0]+1,nxy1[0],nxy1[1]))
            all_paths_count[n] = all_paths_count[curxy] + 2
        else: #note portal
            all_paths_count[n] = all_paths_count[curxy] + 1
        nextxy.put(n)
all_paths_count[end_o]